### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory, DataTypeStr

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

adarDamageSeqDataDirectory = os.path.join(getDataDir(), "Adar_damage-seq")
adarDamageSeqCellularBasename = "A549_BPDE_cell_2h"
adarDamageSeqNakedBasename = "A549_BPDE_nDNA_2h"

adarDamageSeqCellularMutperiodInputFilePaths = getFilesInDirectory(os.path.join(adarDamageSeqDataDirectory, adarDamageSeqCellularBasename), DataTypeStr.mutations + ".bed")
adarDamageSeqNakedMutperiodInputFilePath = getFilesInDirectory(os.path.join(adarDamageSeqDataDirectory, adarDamageSeqNakedBasename), DataTypeStr.mutations + ".bed", searchRecursively = False)

### Run the mutperiod pipeline to count damaged bases relative to transcription factor binding sites.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite, generateCustomBackground

TFBS_FilePaths = list()
TFBS_FilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_CTCF_known", "hg19_CTCF_known.bed"))
# TFBS_FilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_ETS_and_GABPA_known", "hg19_ETS_and_GABPA_known.bed"))
TFBS_FilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_SP1_known", "hg19_SP1_known.bed"))

TFBS_Names = [os.path.basename(TFBS_FilePath).rsplit('.', 1)[0] for TFBS_FilePath in TFBS_FilePaths]

generateCustomBackground(os.path.dirname(adarDamageSeqNakedMutperiodInputFilePath), TFBS_Names,
                         useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)

runAnalysisSuite(adarDamageSeqCellularMutperiodInputFilePaths, TFBS_Names,
                 normalizationMethod = "Custom Background", customBackgroundDir = os.path.dirname(adarDamageSeqNakedMutperiodInputFilePath),
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)